<a href="https://colab.research.google.com/github/ayodelenaomii/ayodelenaomii/blob/main/Titanic_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import zipfile

# Open the zip file
with zipfile.ZipFile('/content/drive/MyDrive/Titanic Data/titanic.zip', 'r') as zip_ref:
    # Extract the CSV file from the zip
    with zip_ref.open('train.csv') as file:
        Data_train = pd.read_csv(file)
    with zip_ref.open('test.csv') as file:
        Data_test = pd.read_csv(file)
    with zip_ref.open('gender_submission.csv') as file:
        Data_gender = pd.read_csv(file)


print(Data_test.head())

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


EDA


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report





In [ ]:
def clean(data):
    # Drop unnecessary columns
    columns_to_drop = ['Name', 'Cabin', 'PassengerId', 'Ticket']
    data = data.drop(columns=[col for col in columns_to_drop if col in data.columns], axis=1)

    cols = ['Age', 'SibSp', 'Parch', 'Fare']
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)

    data.Embarked.fillna('U', inplace=True)
    data.dropna(inplace=True)

    return data

datatrain = clean(Data_train)
datatest = clean(Data_test)

datatrain.head()


<ipython-input-4-d33ce02ed45d>:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
<ipython-input-4-d33ce02ed45d>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
columns = ["Sex", "Embarked"]

for col in columns:
    datatrain[col] = le.fit_transform(datatrain[col])
    datatest[col] = le.transform(datatest[col])
    print(le.classes_)

datatrain.head()

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [ ]:
y = datatrain["Survived"]
X = datatrain.drop("Survived", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)

accuracy_score(y_test, predictions)

0.8100558659217877

In [ ]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)


predictions = model.predict(X_test)

accuracy_score(y_test, predictions)


0.8212290502793296

In [ ]:
submission_preds = clf.predict(datatest)

In [ ]:
df = pd.DataFrame({'PassengerId': Data_test['PassengerId'], 'Survived': submission_preds})

In [ ]:
df.to_csv('submissiion.csv', index=False)